In [ ]:
import sagemaker
from sagemaker import get_execution_role

In [ ]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()

In [ ]:
%store -r bucket

In [ ]:
training_input_path = f's3://{bucket}/processing_output/train_data'
val_input_path = f's3://{bucket}/processing_output/validation_data'

### Set up Huggingface training job

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 2,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased'
                 }

In [ ]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            output_path=f's3://{bucket}/training_output/',
                            base_job_name="az-ade-training",
                            hyperparameters = hyperparameters,
                            disable_profiler=True)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'val': val_input_path})

### Save training job name for next session

In [ ]:
training_job_name = huggingface_estimator.latest_training_job.name
training_job_name

In [ ]:
%store training_job_name